In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import cifar10

In [2]:
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

In [3]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [4]:
train_y_eye = np.eye(10)[train_y]
test_y_eye = np.eye(10)[test_y]

In [5]:
train_x[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [6]:
train_x = train_x/255.
test_x = test_x/255.

In [7]:
train_x[0]

array([[[0.23137255, 0.24313725, 0.24705882],
        [0.16862745, 0.18039216, 0.17647059],
        [0.19607843, 0.18823529, 0.16862745],
        ...,
        [0.61960784, 0.51764706, 0.42352941],
        [0.59607843, 0.49019608, 0.4       ],
        [0.58039216, 0.48627451, 0.40392157]],

       [[0.0627451 , 0.07843137, 0.07843137],
        [0.        , 0.        , 0.        ],
        [0.07058824, 0.03137255, 0.        ],
        ...,
        [0.48235294, 0.34509804, 0.21568627],
        [0.46666667, 0.3254902 , 0.19607843],
        [0.47843137, 0.34117647, 0.22352941]],

       [[0.09803922, 0.09411765, 0.08235294],
        [0.0627451 , 0.02745098, 0.        ],
        [0.19215686, 0.10588235, 0.03137255],
        ...,
        [0.4627451 , 0.32941176, 0.19607843],
        [0.47058824, 0.32941176, 0.19607843],
        [0.42745098, 0.28627451, 0.16470588]],

       ...,

       [[0.81568627, 0.66666667, 0.37647059],
        [0.78823529, 0.6       , 0.13333333],
        [0.77647059, 0

In [8]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [9]:
n_batch = 100
batch_size = int(len(train_x)/n_batch)
epoch = 200


In [11]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape,

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [12]:
valid = int(0.9*len(train_x))

valid_x = train_x[valid:]
valid_y = train_y[valid:]

train_x = train_x[:valid]
train_y = train_y[:valid]

valid_y = keras.utils.to_categorical(valid_y, 10)
train_y = keras.utils.to_categorical(train_y, 10)
test_y = keras.utils.to_categorical(test_y, 10)

In [13]:
train_x.shape, test_x.shape, valid_x.shape, train_y.shape, test_y.shape, valid_y.shape

((45000, 32, 32, 3),
 (10000, 32, 32, 3),
 (5000, 32, 32, 3),
 (45000, 10),
 (10000, 10),
 (5000, 10))

In [14]:
with tf.device('device:GPU:1'):
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    opt = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(train_x, train_y, batch_size=n_batch, epochs=epoch, validation_data=(valid_x, valid_y), shuffle=True)
    
    scores = model.evaluate(test_x, test_y, verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 45000 samples, validate on 5000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
45000/45000 [==============================] - 267s 6ms/sample - loss: 1.6476 - acc: 0.3929 - val_loss: 1.2591 - val_acc: 0.5440
Epoch 2/200
45000/45000 [==============================] - 262s 6ms/sample - loss: 1.2379 - acc: 0.5563 - val_loss: 1.0317 - val_acc: 0.6360
Epoch 3/200
45000/45000 [==============================] - 271s 6ms/sample - loss: 1.0629 - acc: 0.6242 - val_loss: 0.9590 - val_acc: 0.6656
Epoch 4/200
45000/45000 [==============================] - 263s 6ms/sample - loss: 0.9498 - acc: 0.6668 - val_loss: 0.8536 - val_acc: 0.7090
Epoch 5/200
45000/45000 [==============================] - 268s 6ms/sample - loss: 0.8623 - acc: 0.6966 - val_loss: 0.7548 - val_acc: 0.7418
Epoch 6/200

45000/45000 [==============================] - 122s 3ms/sample - loss: 0.2666 - acc: 0.9064 - val_loss: 0.6389 - val_acc: 0.8172
Epoch 52/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.2558 - acc: 0.9106 - val_loss: 0.6686 - val_acc: 0.8148
Epoch 53/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.2607 - acc: 0.9098 - val_loss: 0.6370 - val_acc: 0.8168
Epoch 54/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.2606 - acc: 0.9090 - val_loss: 0.6442 - val_acc: 0.8124
Epoch 55/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.2568 - acc: 0.9098 - val_loss: 0.6491 - val_acc: 0.8138
Epoch 56/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.2487 - acc: 0.9118 - val_loss: 0.6374 - val_acc: 0.8192
Epoch 57/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.2476 - acc: 0.9124 - val_loss: 0.6396 - val_acc: 0.8184
Epoch 58/200
45000/

Epoch 109/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1947 - acc: 0.9345 - val_loss: 0.7175 - val_acc: 0.8156
Epoch 110/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1906 - acc: 0.9346 - val_loss: 0.6872 - val_acc: 0.8202
Epoch 111/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1860 - acc: 0.9357 - val_loss: 0.7199 - val_acc: 0.8224
Epoch 112/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1864 - acc: 0.9351 - val_loss: 0.6918 - val_acc: 0.8174
Epoch 113/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1914 - acc: 0.9337 - val_loss: 0.6879 - val_acc: 0.8210
Epoch 114/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1858 - acc: 0.9366 - val_loss: 0.7190 - val_acc: 0.8168
Epoch 115/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1840 - acc: 0.9361 - val_loss: 0.7352 - val_acc: 0.8142

45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1671 - acc: 0.9431 - val_loss: 0.7812 - val_acc: 0.8084
Epoch 167/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1638 - acc: 0.9461 - val_loss: 0.7607 - val_acc: 0.8182
Epoch 168/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1697 - acc: 0.9422 - val_loss: 0.7889 - val_acc: 0.8142
Epoch 169/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1672 - acc: 0.9429 - val_loss: 0.7748 - val_acc: 0.8174
Epoch 170/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1600 - acc: 0.9454 - val_loss: 0.7778 - val_acc: 0.8200
Epoch 171/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1627 - acc: 0.9453 - val_loss: 0.7854 - val_acc: 0.8192
Epoch 172/200
45000/45000 [==============================] - 122s 3ms/sample - loss: 0.1646 - acc: 0.9440 - val_loss: 0.7656 - val_acc: 0.8176
Epoch 173/200

In [ ]:
model.fit?

In [19]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix